In [1]:
from gensim.models.fasttext import FastText, save_facebook_model
import pandas as pd
import jieba
import numpy as np
import joblib
import re
from SATG import SeqTheta2

`pip install SATG-0.1-py3-none-any.whl`

# Dealing with the FastText

| aim | training the fasttext models |
| :-: | :- |
| Chinese model | https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.zh.300.bin.gz |
| English model | https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz |

`rmSm_chinese_dataorigin10.csv` is the open-source dataset for TTPs classification
`./SATG_pretrained_models/` keeps the pretrained models
`./_data/` is where user keeps data. Now is empty. You need **First Download the FastText models above** to init the environment. 

In [ ]:
model = FastText.load_fasttext_format('./_data/cc.zh.300.bin')

In [10]:
datas = pd.read_csv('./rmSm_chinese_dataorigin10.csv', encoding='gbk')

In [ ]:
texts = datas['Text'].values.tolist()

In [ ]:
words = [[w for w in l if w not in ' .,()-&%/\\:><@\'"'] for l in [jieba.lcut(t.lower()) for t in texts]]

In [ ]:
model.build_vocab(words, update=True)

In [ ]:
model.train(words, total_examples=len(words), epochs=10)

In [ ]:
save_facebook_model(model, './_data/ttp_fasttext_model_cn.bin')

In [ ]:
vecshape = (len(words), np.max(np.array([len(l) for l in words])), model.wv[words[0][1]].shape[0])

In [ ]:
vecs = np.zeros(vecshape, dtype='float32')
for i, l in enumerate(words):
    for j, w in enumerate(l):
        vecs[i,j] = model.wv[w]

In [ ]:
joblib.dump((words, vecs), './_data/ttp_fasttext_words_vecs_cn.data')

In [ ]:
def cut_train_test(k=10, words=words, vecs=vecs, datas=datas, save_path='./_data/dataorigin10/cn'):
    indexs = [[i for i in range(len(words)) if i%k==s] for s in range(k)]
    tac_ids = [k for k in datas.keys() if re.match('TA\d+', k)]
    tec_ids = [k for k in datas.keys() if re.match('T\d+', k)]
    tac_vecs = datas[tac_ids].values
    tec_vecs = datas[tec_ids].values
    w = np.array(words)
    for i, test_index in enumerate(indexs):
        train_index = []
        for j in range(len(indexs)):
            if i != j:
                train_index += indexs[j]
        train_words = w[train_index].tolist()
        test_words = w[test_index].tolist()
        train_sets = [vecs[train_index], tac_vecs[train_index], tec_vecs[train_index]]
        test_sets = [vecs[test_index], tac_vecs[test_index], tec_vecs[test_index]]
        save_file = save_path+'-k='+str(k)+'-test='+str(i)+'.data'
        joblib.dump((train_sets, None, test_sets, train_index, None, test_index, train_words, None, test_words), save_file)
        print(save_file,'complete!')

In [ ]:
cut_train_test(10)

# Retrain SATG's models 
 We have 2 types of models:
 - SeqTheta: the model with only one topic prototype
 - MultiSeq: the model which holds more than one topic prototypes

In [ ]:
test = joblib.load('./_data/dataorigin10/cn-k=10-test=0.data')

In [ ]:
OUTINDEX = 2

In [ ]:
seqtheta2 = SeqTheta2(
    in_shape=test[0][0].shape[1:],
    out_shape=test[0][OUTINDEX].shape[-1],
    
    use_birnn=False,
    use_conv=True,
    f_type='dense',
    
    score_l=2,#l范式
    theta=0.,#开始阈值:σ(theta)
    cut_scale=None,#放大倍率
    cut_lambda=0.01,#拉普拉斯平滑系数
    use_top_k=False,
    activation='sigmoid',
    
    test_datas=[
        test[2][0],
        test[2][OUTINDEX]
    ],
)

In [ ]:
seqtheta2.train(
    x=test[0][0],
    y=test[0][OUTINDEX],
    z=np.zeros(test[0][OUTINDEX].shape),
    epochs=40,
    batch_size=64,
    
    use_model='notheta4neg',
    
    use_test_epoch=True
)

In [ ]:
seqtheta2.evaluate(
    test[2][0],
    test[2][OUTINDEX]
)

In [ ]:
seqtheta2.save_all_out_model('./SATG_pretrained_models/seqtheta3-new-fasttext-cn.model')

# How to use pretrained models to predict TTPs in CTIs

 We still provide two pre-trained models for Chinese deal as well as English. 
 
 They can be found in dict `./SeqMask_theta_models/`

In [2]:
def getWords(text, split_words="\n.。", line_max=80):
    words = []
    sents = text.split(split_words[0])
    for sent in sents:
        ws = jieba.lcut(sent.lower())
        if len(ws) >= line_max:
            ws = getWords(ws, split_words=split_words[1:], line_max=line_max)
            words += ws
        else:
            words.append(ws)
    return words

In [3]:
def changeVecs(words, fastText, line_max=80, vec_max=300):
    shape = (len(words), line_max, vec_max)
    vecs = np.zeros(shape, dtype='float32')
    for i, line in enumerate(words):
        for j, w in enumerate(line):
            vecs[i,j] = fastText.wv[w]
    return vecs

In [26]:
def print_predict(SeqTheta, vecs):
    outs = SeqTheta.score_model.predict(vecs)
    print("words-score:", outs[0])
    print("words-score-percent:", outs[1])
    print("positive ttp predict:", outs[2])
    print("negative ttp predict:", outs[3])
    print("original ttp predict:", outs[3])

In [4]:
tactics_labels = ['TA0009', 'TA0010', 'TA0011', 'TA0004', 'TA0005', 'TA0001', 'TA0006', 'TA0007', 'TA0003', 'TA0040', 'TA0002', 'TA0008']
technique_labels = [
    'T1220', 'T1132', 'T1204', 'T1585', 'T1485', 'T1008', 'T1040', 'T1496', 'T1437', 'T1410', 'T1218', 'T1025', 'T1205', 'T1041',
    'T1446', 'T1555', 'T1560', 'T1577', 'T1074', 'T1080', 'T1200', 'T1489', 'T1491', 'T1012', 'T1583', 'T1516', 'T1407', 'T1484', 
    'T1055', 'T1438', 'T1517', 'T1112', 'T1430', 'T1552', 'T1072', 'T1409', 'T1400', 'T1187', 'T1001', 'T1052', 'T1518', 'T1203', 
    'T1521', 'T1554', 'T1587', 'T1528', 'T1547', 'T1413', 'T1499', 'T1558', 'T1135', 'T1102', 'T1119', 'T1507', 'T1571', 'T1570', 
    'T1195', 'T1557', 'T1543', 'T1562', 'T1201', 'T1114', 'T1486', 'T1480', 'T1490', 'T1140', 'T1420', 'T1512', 'T1542', 'T1020', 
    'T1090', 'T1452', 'T1046', 'T1414', 'T1120', 'T1213', 'T1217', 'T1474', 'T1402', 'T1458', 'T1003', 'T1417', 'T1401', 'T1129', 
    'T1534', 'T1083', 'T1539', 'T1037', 'T1071', 'T1433', 'T1564', 'T1411', 'T1509', 'T1011', 'T1540', 'T1541', 'T1059', 'T1211', 
    'T1115', 'T1016', 'T1027', 'T1398', 'T1418', 'T1030', 'T1010', 'T1561', 'T1199', 'T1189', 'T1482', 'T1559', 'T1098', 'T1436', 
    'T1222', 'T1447', 'T1124', 'T1007', 'T1053', 'T1057', 'T1105', 'T1576', 'T1216', 'T1565', 'T1113', 'T1432', 'T1125', 'T1498', 
    'T1185', 'T1197', 'T1406', 'T1505', 'T1137', 'T1497', 'T1574', 'T1588', 'T1036', 'T1092', 'T1601', 'T1219', 'T1033', 'T1422', 
    'T1444', 'T1070', 'T1520', 'T1569', 'T1018', 'T1039', 'T1568', 'T1190', 'T1456', 'T1471', 'T1068', 'T1529', 'T1106', 'T1544',
    'T1475', 'T1056', 'T1531', 'T1563', 'T1048', 'T1082', 'T1581', 'T1556', 'T1550', 'T1584', 'T1123', 'T1513', 'T1533', 'T1567', 
    'T1087', 'T1005', 'T1111', 'T1546', 'T1133', 'T1207', 'T1428', 'T1021', 'T1448', 'T1047', 'T1136', 'T1478', 'T1481', 'T1566', 
    'T1426', 'T1110', 'T1582', 'T1029', 'T1404', 'T1548', 'T1412', 'T1523', 'T1127', 'T1472', 'T1091', 'T1014', 'T1104', 'T1221', 
    'T1424', 'T1176', 'T1202', 'T1069', 'T1553', 'T1575', 'T1573', 'T1210', 'T1508', 'T1477', 'T1476', 'T1532', 'T1078', 'T1134', 
    'T1095', 'T1572', 'T1429', 'T1435', 'T1049', 'T1579'
]

## Dealing with Chinese

In [5]:
fastText = FastText.load_fasttext_format('./_data/ttp_fasttext_model_cn.bin')

<ipython-input-5-9186f4547003>:2: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  fastText = FastText.load_fasttext_format('./TTP_FastText/ttp_fasttext_model_cn.bin')


In [6]:
test_text = "2014年和2015年报告的bisonal变体在c2中使用了一个简单的xor密码。一些bisonal样本用rc4加密c2通信。"

In [7]:
test_words = getWords(test_text, line_max=79)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.576 seconds.
Prefix dict has been built successfully.


In [8]:
test_vecs = changeVecs(test_words, fastText, line_max=79)

In [9]:
testModel = SeqTheta2(in_shape=test_vecs.shape[1:], out_shape=test_vecs.shape[1])
# if you want to use our pretrained models:
testModel.load_models('./SATG_pretrained_models/seqtheta3-new-fasttext-cn.model')

f_type: cnn
f_type: cnn
f_type: cnn
build text score model:
use outs: ['s', 'ps', 'po', 'no', 'to']
build evaluate model:
use outs: ['to', 'po', 'tpo']
build conv vectors out:
use outs: ['c']
build text score model:
use outs: ['s', 'ps', 'po', 'no', 'to']
build evaluate model:
use outs: ['to', 'po', 'tpo']
build conv vectors out:
use outs: ['c']


In [10]:
testModel.print_predict(test_vecs, test_words, technique_labels)

1/1 [==============================] - 3s 3s/step
words score:


Words:,2014,年,和,2015,年,报告,的,bisonal,变体,在,c2,中,使用,了,一个,简单,的,xor,密码,。,一些,bisonal,样本,用,rc4,加密,c2,通信,。
Score:,-4.6376,-7.9106,-12.7448,-4.6158,-7.9106,-4.4181,-16.8811,-4.5591,-6.591,-11.86,-4.5812,-9.669,-13.3025,-12.3278,-12.9407,-5.3558,-16.8811,-4.5879,-4.5582,-12.6032,-8.6173,-4.5591,-4.4313,-9.3852,-4.3902,-4.5633,-4.5812,-4.5794,-12.6032
Sigma:,0.5707,0.0,0.0,0.5899,0.0,0.7462,0.0,0.6385,0.0011,0.0,0.6198,0.0,0.0,0.0,0.0,0.0901,0.0,0.6141,0.6392,0.0,0.0,0.6385,0.7371,0.0,0.7649,0.6349,0.6198,0.6213,0.0


ttp predict:
f(X+): []
f(X): ['T1573-0.6392']


TTP Lable:,T1220,T1132,T1204,T1585,T1485,T1008,T1040,T1496,T1437,T1410,T1218,T1025,T1205,T1041,T1446,T1555,T1560,T1577,T1074,T1080,T1200,T1489,T1491,T1012,T1583,T1516,T1407,T1484,T1055,T1438,T1517,T1112,T1430,T1552,T1072,T1409,T1400,T1187,T1001,T1052,T1518,T1203,T1521,T1554,T1587,T1528,T1547,T1413,T1499,T1558,T1135,T1102,T1119,T1507,T1571,T1570,T1195,T1557,T1543,T1562,T1201,T1114,T1486,T1480,T1490,T1140,T1420,T1512,T1542,T1020,T1090,T1452,T1046,T1414,T1120,T1213,T1217,T1474,T1402,T1458,T1003,T1417,T1401,T1129,T1534,T1083,T1539,T1037,T1071,T1433,T1564,T1411,T1509,T1011,T1540,T1541,T1059,T1211,T1115,T1016,T1027,T1398,T1418,T1030,T1010,T1561,T1199,T1189,T1482,T1559,T1098,T1436,T1222,T1447,T1124,T1007,T1053,T1057,T1105,T1576,T1216,T1565,T1113,T1432,T1125,T1498,T1185,T1197,T1406,T1505,T1137,T1497,T1574,T1588,T1036,T1092,T1601,T1219,T1033,T1422,T1444,T1070,T1520,T1569,T1018,T1039,T1568,T1190,T1456,T1471,T1068,T1529,T1106,T1544,T1475,T1056,T1531,T1563,T1048,T1082,T1581,T1556,T1550,T1584,T1123,T1513,T1533,T1567,T1087,T1005,T1111,T1546,T1133,T1207,T1428,T1021,T1448,T1047,T1136,T1478,T1481,T1566,T1426,T1110,T1582,T1029,T1404,T1548,T1412,T1523,T1127,T1472,T1091,T1014,T1104,T1221,T1424,T1176,T1202,T1069,T1553,T1575,T1573,T1210,T1508,T1477,T1476,T1532,T1078,T1134,T1095,T1572,T1429,T1435,T1049,T1579
f(X+):,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0803,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0066,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0025,0.0,0.0778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
f(X):,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.048,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0015,0.0,0.6392,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0


## Dealing with English

In [11]:
fastText = FastText.load_fasttext_format('./_data/ttp_fasttext_model.bin')

<ipython-input-11-21aa432b18fd>:1: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  fastText = FastText.load_fasttext_format('./TTP_FastText/ttp_fasttext_model.bin')


In [12]:
test_text = """The encoded VBScript uses a file extension which is not associated, by default, as being a VBScript file.
The extensions .domx and .lgx have been observed. The shellcode is responsible for adding a new file
association for the file extension which specifies that they should be interpreted as an encoded VBScript.
Finally, the shellcode executes the encoded VBScript file which will extract the encrypted files from the
binary blob, show the decoy document (if there is one), and execute the malware."""

In [13]:
test_words = getWords(test_text, line_max=80)
test_vecs = changeVecs(test_words, fastText, line_max=80)

In [15]:
testModel = SeqTheta2(in_shape=test_vecs.shape[1:], out_shape=test_vecs.shape[1])
# if you want to use our pretrained models:
testModel.load_models('./SATG_pretrained_models/seqtheta2-new-fasttext.model')

f_type: cnn
f_type: cnn
f_type: cnn
build text score model:
use outs: ['s', 'ps', 'po', 'no', 'to']
build evaluate model:
use outs: ['to', 'po', 'tpo']
build conv vectors out:
use outs: ['c']
build text score model:
use outs: ['s', 'ps', 'po', 'no', 'to']
build evaluate model:
use outs: ['to', 'po', 'tpo']
build conv vectors out:
use outs: ['c']


In [16]:
testModel.print_predict(test_vecs, test_words, technique_labels)

1/1 [==============================] - 0s 332ms/step
words score:


Words:,the,,encoded,,vbscript,,uses,,a,,file,,extension,,which,,is,,not,,associated,",",,by,,default,",",,as,,being,,a,,vbscript,,file,.
Score:,-7.0489,-2.9225,-2.9272,-2.9225,-2.7812,-2.9225,-3.7225,-2.9225,-15.23,-2.9225,-2.9534,-2.9225,-2.9855,-2.9225,-3.7532,-2.9225,-13.9996,-2.9225,-4.939,-2.9225,-3.1921,-5.4322,-2.9225,-3.3545,-2.9225,-2.8743,-5.4322,-2.9225,-9.9225,-2.9225,-3.7606,-2.9225,-15.23,-2.9225,-2.7812,-2.9225,-2.9534,-2.9619
Sigma:,0.0,0.7426,0.7398,0.7426,0.8184,0.7426,0.1875,0.7426,0.0,0.7426,0.7235,0.7426,0.7028,0.7426,0.1732,0.7426,0.0,0.7426,0.0049,0.7426,0.5518,0.001,0.7426,0.4245,0.7426,0.7706,0.001,0.7426,0.0,0.7426,0.1698,0.7426,0.0,0.7426,0.8184,0.7426,0.7235,0.7181


ttp predict:
f(X+): ['T1027-0.6214']
f(X): ['T1027-0.6126']


TTP Lable:,T1220,T1132,T1204,T1585,T1485,T1008,T1040,T1496,T1437,T1410,T1218,T1025,T1205,T1041,T1446,T1555,T1560,T1577,T1074,T1080,T1200,T1489,T1491,T1012,T1583,T1516,T1407,T1484,T1055,T1438,T1517,T1112,T1430,T1552,T1072,T1409,T1400,T1187,T1001,T1052,T1518,T1203,T1521,T1554,T1587,T1528,T1547,T1413,T1499,T1558,T1135,T1102,T1119,T1507,T1571,T1570,T1195,T1557,T1543,T1562,T1201,T1114,T1486,T1480,T1490,T1140,T1420,T1512,T1542,T1020,T1090,T1452,T1046,T1414,T1120,T1213,T1217,T1474,T1402,T1458,T1003,T1417,T1401,T1129,T1534,T1083,T1539,T1037,T1071,T1433,T1564,T1411,T1509,T1011,T1540,T1541,T1059,T1211,T1115,T1016,T1027,T1398,T1418,T1030,T1010,T1561,T1199,T1189,T1482,T1559,T1098,T1436,T1222,T1447,T1124,T1007,T1053,T1057,T1105,T1576,T1216,T1565,T1113,T1432,T1125,T1498,T1185,T1197,T1406,T1505,T1137,T1497,T1574,T1588,T1036,T1092,T1601,T1219,T1033,T1422,T1444,T1070,T1520,T1569,T1018,T1039,T1568,T1190,T1456,T1471,T1068,T1529,T1106,T1544,T1475,T1056,T1531,T1563,T1048,T1082,T1581,T1556,T1550,T1584,T1123,T1513,T1533,T1567,T1087,T1005,T1111,T1546,T1133,T1207,T1428,T1021,T1448,T1047,T1136,T1478,T1481,T1566,T1426,T1110,T1582,T1029,T1404,T1548,T1412,T1523,T1127,T1472,T1091,T1014,T1104,T1221,T1424,T1176,T1202,T1069,T1553,T1575,T1573,T1210,T1508,T1477,T1476,T1532,T1078,T1134,T1095,T1572,T1429,T1435,T1049,T1579
f(X+):,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0003,0.0,0.0,0.0001,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0002,0.0,0.0,0.0,0.0,0.0013,0.0,0.0011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0004,0.0,0.0022,0.0,0.0,0.0,0.0,0.0,0.0028,0.0,0.0,0.0044,0.0,0.0,0.0,0.6214,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0003,0.0,0.0,0.0004,0.0009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0006,0.0,0.0,0.0,0.0004,0.0,0.0,0.0,0.0,0.0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0066,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0003,0.0,0.0,0.0,0.0028,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0002,0.0,0.0,0.0,0.0,0.0
f(X):,0.0,0.0014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0053,0.0,0.0,0.0,0.0,0.0,0.0,0.0013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0002,0.0,0.0,0.0,0.0,0.0002,0.0,0.0013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0009,0.0,0.0,0.0001,0.0,0.0,0.0005,0.0,0.0,0.0004,0.0,0.0,0.0,0.6126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0002,0.0,0.0,0.0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0003,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0014,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


words score:


...

TTP Lable:,T1220,T1132,T1204,T1585,T1485,T1008,T1040,T1496,T1437,T1410,T1218,T1025,T1205,T1041,T1446,T1555,T1560,T1577,T1074,T1080,T1200,T1489,T1491,T1012,T1583,T1516,T1407,T1484,T1055,T1438,T1517,T1112,T1430,T1552,T1072,T1409,T1400,T1187,T1001,T1052,T1518,T1203,T1521,T1554,T1587,T1528,T1547,T1413,T1499,T1558,T1135,T1102,T1119,T1507,T1571,T1570,T1195,T1557,T1543,T1562,T1201,T1114,T1486,T1480,T1490,T1140,T1420,T1512,T1542,T1020,T1090,T1452,T1046,T1414,T1120,T1213,T1217,T1474,T1402,T1458,T1003,T1417,T1401,T1129,T1534,T1083,T1539,T1037,T1071,T1433,T1564,T1411,T1509,T1011,T1540,T1541,T1059,T1211,T1115,T1016,T1027,T1398,T1418,T1030,T1010,T1561,T1199,T1189,T1482,T1559,T1098,T1436,T1222,T1447,T1124,T1007,T1053,T1057,T1105,T1576,T1216,T1565,T1113,T1432,T1125,T1498,T1185,T1197,T1406,T1505,T1137,T1497,T1574,T1588,T1036,T1092,T1601,T1219,T1033,T1422,T1444,T1070,T1520,T1569,T1018,T1039,T1568,T1190,T1456,T1471,T1068,T1529,T1106,T1544,T1475,T1056,T1531,T1563,T1048,T1082,T1581,T1556,T1550,T1584,T1123,T1513,T1533,T1567,T1087,T1005,T1111,T1546,T1133,T1207,T1428,T1021,T1448,T1047,T1136,T1478,T1481,T1566,T1426,T1110,T1582,T1029,T1404,T1548,T1412,T1523,T1127,T1472,T1091,T1014,T1104,T1221,T1424,T1176,T1202,T1069,T1553,T1575,T1573,T1210,T1508,T1477,T1476,T1532,T1078,T1134,T1095,T1572,T1429,T1435,T1049,T1579
f(X+):,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0001,0.0,0.0001,0.0,0.0,0.0,0.0,0.0009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0052,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0153,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.8766,0.0,0.0,0.0,0.2169,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002,0.0,0.0024,0.0,0.0,0.0,0.0,0.0,0.0,0.0002,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005,0.0,0.0,0.0,0.0,0.0,0.0,0.0002,0.0,0.0003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0002,0.0,0.0,0.0,0.0003,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0
f(X):,0.0,0.0011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0004,0.0,0.0,0.0,0.0,0.0,0.0,0.0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0002,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0003,0.0,0.0,0.0,0.2065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0


words score:


Words:,binary,,blob,",",,show,,the,,decoy,,document,,(,if,,there,,is,,one,),",",,and,,execute,,the,,malware,.
Score:,-2.9265,-2.9225,-3.3131,-5.4322,-2.9225,-3.3285,-2.9225,-7.0489,-2.9225,-2.8429,-2.9225,-3.0664,-2.9225,-3.4268,-11.3149,-2.9225,-3.8387,-2.9225,-13.9996,-2.9225,-4.4119,-2.9409,-5.4322,-2.9225,-5.7416,-2.9225,-3.5395,-2.9225,-7.0489,-2.9225,-3.1245,-2.9619
Sigma:,0.7371,0.7397,0.4297,0.0006,0.7397,0.4169,0.7397,0.0,0.7397,0.7883,0.7397,0.6354,0.7397,0.3386,0.0,0.7397,0.1121,0.7397,0.0,0.7397,0.0177,0.7275,0.0006,0.7397,0.0002,0.7397,0.2587,0.7397,0.0,0.7397,0.5885,0.7131


ttp predict:
f(X+): []
f(X): []


TTP Lable:,T1220,T1132,T1204,T1585,T1485,T1008,T1040,T1496,T1437,T1410,T1218,T1025,T1205,T1041,T1446,T1555,T1560,T1577,T1074,T1080,T1200,T1489,T1491,T1012,T1583,T1516,T1407,T1484,T1055,T1438,T1517,T1112,T1430,T1552,T1072,T1409,T1400,T1187,T1001,T1052,T1518,T1203,T1521,T1554,T1587,T1528,T1547,T1413,T1499,T1558,T1135,T1102,T1119,T1507,T1571,T1570,T1195,T1557,T1543,T1562,T1201,T1114,T1486,T1480,T1490,T1140,T1420,T1512,T1542,T1020,T1090,T1452,T1046,T1414,T1120,T1213,T1217,T1474,T1402,T1458,T1003,T1417,T1401,T1129,T1534,T1083,T1539,T1037,T1071,T1433,T1564,T1411,T1509,T1011,T1540,T1541,T1059,T1211,T1115,T1016,T1027,T1398,T1418,T1030,T1010,T1561,T1199,T1189,T1482,T1559,T1098,T1436,T1222,T1447,T1124,T1007,T1053,T1057,T1105,T1576,T1216,T1565,T1113,T1432,T1125,T1498,T1185,T1197,T1406,T1505,T1137,T1497,T1574,T1588,T1036,T1092,T1601,T1219,T1033,T1422,T1444,T1070,T1520,T1569,T1018,T1039,T1568,T1190,T1456,T1471,T1068,T1529,T1106,T1544,T1475,T1056,T1531,T1563,T1048,T1082,T1581,T1556,T1550,T1584,T1123,T1513,T1533,T1567,T1087,T1005,T1111,T1546,T1133,T1207,T1428,T1021,T1448,T1047,T1136,T1478,T1481,T1566,T1426,T1110,T1582,T1029,T1404,T1548,T1412,T1523,T1127,T1472,T1091,T1014,T1104,T1221,T1424,T1176,T1202,T1069,T1553,T1575,T1573,T1210,T1508,T1477,T1476,T1532,T1078,T1134,T1095,T1572,T1429,T1435,T1049,T1579
f(X+):,0.0003,0.0,0.0,0.0,0.0004,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0007,0.0,0.0,0.0,0.0,0.0,0.0059,0.0036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0005,0.0,0.0,0.0,0.0222,0.0003,0.0,0.023,0.0,0.0001,0.0,0.0001,0.0,0.0,0.0001,0.0,0.0,0.0007,0.0047,0.0,0.0001,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0001,0.0009,0.0001,0.0,0.0,0.0,0.0,0.0005,0.0002,0.0,0.0016,0.0,0.0001,0.0006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0002,0.0,0.0,0.0,0.0,0.0028,0.0,0.0,0.0,0.0,0.0,0.0004,0.0001,0.0,0.0,0.0009,0.0,0.0,0.0003,0.0,0.0,0.0001,0.0,0.0,0.002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0006,0.0025,0.0,0.0,0.0,0.0014,0.0014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0004,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0003,0.0,0.0003,0.0,0.0293,0.005,0.0,0.0,0.0007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0
f(X):,0.0002,0.0,0.0,0.0,0.0001,0.0021,0.001,0.0,0.0,0.0,0.0,0.0,0.0002,0.0,0.0,0.0,0.0001,0.0,0.0004,0.0001,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0084,0.0001,0.0,0.0004,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0102,0.0088,0.0,0.0001,0.0,0.0,0.0007,0.0,0.0,0.0,0.0,0.0,0.0008,0.0,0.0,0.0,0.0,0.0,0.0005,0.0,0.0,0.0015,0.0,0.0003,0.0004,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0002,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0001,0.0,0.0,0.0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0021,0.0,0.0001,0.0001,0.0,0.0006,0.0001,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0011,0.0001,0.0,0.0,0.0,0.0003,0.0,0.0005,0.0,0.0,0.0,0.0,0.0,0.0,0.0004,0.0001,0.0,0.0004,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001,0.0,0.0001,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0003,0.0,0.0001,0.0,0.1046,0.0003,0.0,0.0002,0.0001,0.0,0.0,0.0,0.0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
print_predict(testModel, test_vecs)

1/1 [==============================] - 0s 30ms/step
words-score: [[ -7.048914   -2.9225307  -2.9271724  -2.9225307  -2.7811809  -2.9225307
   -3.722545   -2.9225307 -15.229999   -2.9225307  -2.9534054  -2.9225307
   -2.9854913  -2.9225307  -3.7532005  -2.9225307 -13.9995775  -2.9225307
   -4.939002   -2.9225307  -3.1921496  -5.4322085  -2.9225307  -3.3544648
   -2.9225307  -2.874281   -5.4322085  -2.9225307  -9.922531   -2.9225307
   -3.7606065  -2.9225307 -15.229999   -2.9225307  -2.7811809  -2.9225307
   -2.9534054  -2.961901   -2.9225307  -2.9225307  -2.9225307  -2.9225307
   -2.9225307  -2.9225307  -2.9225307  -2.9225307  -2.9225307  -2.9225307
   -2.9225307  -2.9225307  -2.9225307  -2.9225307  -2.9225307  -2.9225307
   -2.9225307  -2.9225307  -2.9225307  -2.9225307  -2.9225307  -2.9225307
   -2.9225307  -2.9225307  -2.9225307  -2.9225307  -2.9225307  -2.9225307
   -2.9225307  -2.9225307  -2.9225307  -2.9225307  -2.9225307  -2.9225307
   -2.9225307  -2.9225307  -2.9225307  -2.92253